## Feature Engineering

En esta etapa del proyecto, se prepararan los conjuntos datos necesarios para las funciones.

#### Importación de librerias necesarias:

In [105]:
import pandas as pd
import numpy as np
from textblob import TextBlob

#### Lectura de los datasets modificados durante la etapa de ETL:

In [106]:
steam_games = pd.read_csv('../steam_games_limpio.csv')

user_reviews = pd.read_csv('../user_reviews_limpio.csv')

user_items = pd.read_csv('../user_items_limpio.csv')

### Sentiment Analysis

Creación de una nueva categoria en el dataset **user_reviews**, utilizando el analisis de sentimientos a traves de procesamiento de lenguaje natural. Esto asignara etiquetas a las opiniones siguiendo el siguiente patron:
- 0 : Se aplicara a opiniones consideradas negativas.
- 1 : Se aplicara a opiniones consideradas neutras.
- 2 : Se aplicara a opiniones consideradas positivas.

En casos donde no sea posible determinar la clasificación debido a la falta de reseña, se utilizara la etiqueta **1** (neutra).

In [107]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   funny        8010 non-null   object 
 1   posted       48498 non-null  float64
 2   last_edited  6037 non-null   object 
 3   item_id      58431 non-null  int64  
 4   helpful      58431 non-null  object 
 5   recommend    58431 non-null  bool   
 6   review       58401 non-null  object 
 7   user_id      58431 non-null  object 
 8   user_url     58431 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(6)
memory usage: 3.6+ MB


Modificación del tipo de dato de la columna **reviews** a string:

In [108]:
user_reviews['review'] = user_reviews['review'].astype(str)

Verificación:

In [109]:
type(user_reviews['review'][0])

str

Creación de la función para poder analizar la columna **review**:

In [110]:
def analizar_sentimiento(opinion):

    if isinstance(opinion, str) and len(opinion) > 0:

        analysis = TextBlob(opinion)
        
        sentiment = analysis.sentiment.polarity

        if sentiment < -0.1:
            return 0  
        
        elif sentiment > 0.1:
            return 2 
        
        else:
            return 1 
        
    else:
        return 1  

In [111]:
user_reviews['sentiment_analysis'] = user_reviews['review'].apply(analizar_sentimiento)

In [112]:
user_reviews.to_csv('../sentiment_analysis.csv', index=False)

Controlamos que el analisis sea el correcto:

In [113]:
user_reviews['review'][3]

'I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it\'s title, this is easily one of my GOTYs. You don\'t get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can\'t 360 noscope your friends, but what you can do is show them up with your bad ♥♥♥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8'

In [114]:
user_reviews['sentiment_analysis'][3]

2

Eliminamos la columna **review**, ya que esta sera reemplazada por la columna **sentiment_analysis** para facilitar el trabajo de los modelos de Machine Learning y el analisis de datos:

In [115]:
user_reviews = user_reviews.drop(columns = 'review')

## Creación de funciones

Ahora, crearemos las funciones para los endpoints que se consumirán en la API

#### Función **PlayTimeGenre**

*def PlayTimeGenre(genero: str):* Debe devolver año con más horas jugadas para dicho género.

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

Cambiamos el nombre del DataFrame **steam_games**, ya que luego usaremos esa columna para hacer merge.

In [116]:
steam_games = steam_games.rename(columns={'id': 'item_id'})

Se extraen las columnas necesarias del DataFrame **steam_games**, estas son:
- genres
- release_year
- item_id

In [117]:
games = steam_games[['genres', 'release_year', 'item_id']]

Se extraen las columnas necesarias del DataFrame **user_items**, estas son:
- item_id
- playtime_forever

In [118]:
items = user_items[['item_id', 'playtime_forever']]

Combinamos los DataFrames para formar uno nuevo con las columnas necesarias para esta función.

In [119]:
items_games = games.merge(items, on="item_id")

Eliminamos la columna **item_id**, ya que no es necesaria para nuestra función. Solo utilizamos esta columna para realizar la combinación.

In [120]:
items_games = items_games.drop(columns = 'item_id')

Volvemos a revisar los tipos de datos para saber como trabajarlos en nuestra función:

In [121]:
items_games

,genres,release_year,playtime_forever
0,Action,1997.0,5
1,Action,1997.0,0
2,Action,1997.0,0
3,Action,1997.0,0
4,Action,1997.0,13
...,...,...,...
9965682,Action,2004.0,0
9965683,Action,2004.0,0
9965684,Action,2004.0,0
9965685,Action,2004.0,9


In [122]:
items_games['genres'] = items_games['genres'].astype(str)

Creamos el archivo del DataFrame en formato parquet:

In [123]:
items_games.to_parquet('Data/df_funcion_1.parquet')

In [124]:
def PlayTimeGenre(genero):
    # Filtra los datos por género
    genre_data = items_games[items_games['genres'].str.contains(genero, case=False, na=False)]

    # Agrupa por año y suma las horas jugadas
    genre_by_year = genre_data.groupby('release_year')['playtime_forever'].sum().reset_index()

    # Encuentra el año con más horas jugadas
    year_with_most_playtime = genre_by_year.loc[genre_by_year['playtime_forever'].idxmax()]

    return {"Año de lanzamiento con más horas jugadas para " + genero: int(year_with_most_playtime['release_year'])}


In [125]:
PlayTimeGenre('Action')


{'Año de lanzamiento con más horas jugadas para Action': 2012}

#### Función **UserForGenre**

*def UserForGenre(genero: str):* Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas : 23}]}

Se extraen las columnas necesarias del DataFrame **steam_games**, estas son:
- genres 
- item_id

In [126]:
generos = steam_games[['genres', 'release_year', 'item_id']]

Se extraen las columnas necesarias del DataFrame **user_items**, estas son:
- user_id 
- playtime_forever
- item_id

In [127]:
usuarios = user_items[['user_id', 'playtime_forever', 'item_id']]

Combinamos los DataFrames para formar uno nuevo con las columnas necesarias para esta función.

In [128]:
df_funcion_2 = generos.merge(usuarios, on="item_id")

In [129]:
df_funcion_2

,genres,release_year,item_id,user_id,playtime_forever
0,Action,1997.0,282010.0,UTNerd24,5
1,Action,1997.0,282010.0,I_DID_911_JUST_SAYING,0
2,Action,1997.0,282010.0,76561197962104795,0
3,Action,1997.0,282010.0,r3ap3r78,0
4,Action,1997.0,282010.0,saint556,13
...,...,...,...,...,...
9965682,Action,2004.0,80.0,76561198273508956,0
9965683,Action,2004.0,80.0,76561198282090798,0
9965684,Action,2004.0,80.0,943525,0
9965685,Action,2004.0,80.0,76561198283312749,9


Eliminamos la columna **item_id**, ya que no es necesaria para nuestra función. Solo utilizamos esta columna para realizar la combinación.

In [130]:
df_funcion_2 = df_funcion_2.drop(columns= 'item_id')

Eliminamos los valores nulos de la columna **release_year**

In [131]:
df_funcion_2.dropna(subset=['release_year'], inplace=True)

Verificamos que hayan sido eliminados viendo valores unicos:

In [132]:
df_funcion_2['release_year'].unique()

array([1997., 1998., 2006., 2005., 2003., 2007., 2002., 2000., 1995.,
       1996., 1994., 2001., 1993., 2004., 1999., 2008., 2009., 1992.,
       1989., 2010., 2011., 2013., 2012., 2014., 2017., 1983., 1984.,
       2015., 2016., 1990., 1988., 1991., 1987., 2018.])

Creamos el archivo del DataFrame en formato parquet:

In [133]:
df_funcion_2.to_parquet('Data/df_funcion_2.parquet')

In [134]:
def UserForGenre(genero):

    # Filtramos los datos de acuerdo al genero
    data_genero = df_funcion_2[df_funcion_2['genres'].str.contains(genero, case=False, na=False)]

    if data_genero.empty:
        return {"message": "No hay datos disponibles para el género especificado."}

    # Agrupamos por usuario y sumamos el total de horas jugadas.
    horas_jugadas_usuario = data_genero.groupby('user_id')['playtime_forever'].sum().reset_index()

    # Encuentramos el usuario con mayor cantidad horas jugadas.
    user_with_most_playtime = horas_jugadas_usuario.loc[horas_jugadas_usuario['playtime_forever'].idxmax()]

    # Filtramos los datos por usuario, y calculamos la acumulación de horas jugadas por año.
    user_data = data_genero[data_genero['user_id'] == user_with_most_playtime['user_id']]
    playtime_by_year = user_data.groupby('release_year')['playtime_forever'].sum().reset_index()

    # Convertimos los datos a un formato de lista de diccionarios y cambiamos los nombres de las claves, para darle el formato de salida esperado.
    playtime_by_year_list = playtime_by_year.rename(columns={'release_year': 'Año', 'playtime_forever': 'Horas'}).to_dict(orient='records')

    result = {
        "Usuario con más horas jugadas para " + genero: user_with_most_playtime['user_id'],
        "Horas jugadas": playtime_by_year_list
    }

    return result

Probamos la función:

In [135]:
UserForGenre('Indie')

{'Usuario con más horas jugadas para Indie': 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': 1999.0, 'Horas': 0},
  {'Año': 2001.0, 'Horas': 11},
  {'Año': 2003.0, 'Horas': 1863},
  {'Año': 2005.0, 'Horas': 0},
  {'Año': 2006.0, 'Horas': 1673},
  {'Año': 2007.0, 'Horas': 1070},
  {'Año': 2008.0, 'Horas': 1366},
  {'Año': 2009.0, 'Horas': 28993},
  {'Año': 2010.0, 'Horas': 21487},
  {'Año': 2011.0, 'Horas': 100155},
  {'Año': 2012.0, 'Horas': 148459},
  {'Año': 2013.0, 'Horas': 169349},
  {'Año': 2014.0, 'Horas': 326927},
  {'Año': 2015.0, 'Horas': 751765},
  {'Año': 2016.0, 'Horas': 815989},
  {'Año': 2017.0, 'Horas': 33887}]}

#### Función **UsersRemmend**

*def UsersRecommend(año: int):* Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Se extraen las columnas necesarias del DataFrame **user_reviews**, estas son:
- posted
- recommend
- sentiment_analysis
- item_id

In [136]:
reviews = user_reviews[['posted', 'recommend', 'sentiment_analysis', 'item_id']]

Se extraen las columnas necesarias del DataFrame **steam_games**, estas son:
- app_name
- item_id

In [137]:
games_3 = steam_games[['app_name', 'item_id']]

games

,genres,release_year,item_id
0,Action,2018.0,761140.0
1,Casual,2018.0,761140.0
2,Indie,2018.0,761140.0
3,Simulation,2018.0,761140.0
4,Strategy,2018.0,761140.0
...,...,...,...
74829,Racing,2018.0,610660.0
74830,Simulation,2018.0,610660.0
74831,Casual,2017.0,658870.0
74832,Indie,2017.0,658870.0


Combinamos los DataFrames para formar uno nuevo con las columnas necesarias para esta función.

In [138]:
reviews_games = games_3.merge(reviews, on="item_id")


Eliminamos la columna **item_id**, ya que no es necesaria para nuestra función. Solo utilizamos esta columna para realizar la combinación.

In [139]:
reviews_games = reviews_games.drop(columns= 'item_id')

Reindexamos:

In [140]:
reviews_games.reset_index(inplace= True, drop= True)

In [141]:
reviews_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127794 entries, 0 to 127793
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   app_name            127794 non-null  object 
 1   posted              105154 non-null  float64
 2   recommend           127794 non-null  bool   
 3   sentiment_analysis  127794 non-null  int64  
dtypes: bool(1), float64(1), int64(1), object(1)
memory usage: 3.0+ MB


In [142]:
reviews_games.to_parquet('Data/df_funcion_3y4.parquet')

In [143]:
def UsersRecommend(año):

    # Filtramos los datos por el año recibido como parametro.
    reviews_year = reviews_games[reviews_games['posted'] == año]

    # Filtramos las reseñas recomendadas y con sentimiento positivo o neutral.
    recommended_games = reviews_year[(reviews_year['recommend'] == True) & (reviews_year['sentiment_analysis'].isin([1, 2]))]

    # Agrupamos por juego y contamos las recomendaciones.
    top_games = recommended_games.groupby('app_name')['recommend'].count().reset_index()

    # Ordenamos los juegos en orden descendente de recomendaciones.
    top_games = top_games.sort_values(by='recommend', ascending=False)

    # Tomamos los 3 juegos principales.
    top_3_games = top_games.head(3)

    # Convertimos los datos en el formato de retorno esperado.
    result = [{"Puesto " + str(i + 1): game} for i, game in enumerate(top_3_games['app_name'])]

    return result

In [144]:
UsersRecommend(2014)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Unturned'},
 {'Puesto 3': 'Rust'}]

#### Función **UsersNotRemmend**

*def UsersNotRecommend(año: int):* Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. 

(reviews.recommend = False y comentarios negativos)

Utilizamos el DataFrame, creado para la función anterior.

In [145]:
def UserNotRecommend(año):
    
    reviews_year = reviews_games[reviews_games['posted'] == año]

    # Filtra las reseñas recomendadas con sentimiento positivo o neutral
    not_recommended_games = reviews_year[(reviews_year['recommend'] == False) & (reviews_year['sentiment_analysis'] == 0)]

    # Agrupa por juego y cuenta las recomendaciones
    top_games = not_recommended_games.groupby('app_name')['recommend'].count().reset_index()

    # Ordena los juegos en orden descendente de recomendaciones
    top_games = top_games.sort_values(by='recommend', ascending=False)

    # Toma los 3 juegos principales
    top_3_games = top_games.head(3)

    # Convierte los datos en el formato de retorno
    result = [{"Puesto " + str(i + 1): game} for i, game in enumerate(top_3_games['app_name'])]

    return result

Probamos la función:

In [146]:
UserNotRecommend(2014)

[{'Puesto 1': 'DayZ'}, {'Puesto 2': 'Unturned'}, {'Puesto 3': 'Robocraft'}]

#### Función **sentiment_analysis**

*def sentiment_analysis(año: int):* Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentran categorizados con un análisis de sentimiento.

Ejemplo de retorno: {Negativo = 182, Neutral = 120, Positivo = 278}

Se extraen las columnas necesarias del DataFrame **steam_games**, estas son:
- release_year
- item_id

In [147]:
games_4 = steam_games[['release_year', 'item_id']]

Se extraen las columnas necesarias del DataFrame **user_reviews**, estas son:
- sentiment_analysis
- item_id

In [148]:
reviews_2 = user_reviews[['sentiment_analysis', 'item_id']]

Combinamos los DataFrames para formar uno nuevo con las columnas necesarias para esta función.

In [149]:
reviews_games_2 = reviews_2.merge(games_4, on = 'item_id')

Eliminamos la columna **item_id**, ya que no es necesaria para nuestra función. Solo utilizamos esta columna para realizar la combinación.

In [150]:
reviews_games_2 = reviews_games_2.drop(columns = 'item_id')

In [151]:
reviews_games_2.dropna(subset=['release_year'], inplace=True)

Convertimos el tipo de dato de la columna release_year a string, ya que asi lo necesitaremos para nuestra funcón. Si bien nuestro parametro es de tipo int, lo convertiremos para compararlo y hacer la agrupación correspondiente.

In [152]:
reviews_games_2['release_year'] = reviews_games_2['release_year'].astype(int)

In [153]:
reviews_games_2.to_parquet('Data/df_funcion_5.parquet')

In [154]:
def sentiment_analysis(año):
    
    # Filtra los datos por el año dado
    reviews_year = reviews_games_2[reviews_games_2['release_year'] == int(año)]

    Negativos = 0
    Neutral = 0
    Positivos = 0

    for i in reviews_year["sentiment_analysis"]:
        if i == 0:
            Negativos += 1
        elif i == 1:
            Neutral += 1 
        elif i == 2:
            Positivos += 1

    result = {"Negative": Negativos , "Neutral" : Neutral, "Positive": Positivos}
    return result

In [155]:
sentiment_analysis(2014)

{'Negative': 2686, 'Neutral': 7470, 'Positive': 7483}

In [156]:
reviews_games_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 124368 entries, 0 to 127793
Data columns (total 2 columns):
 #   Column              Non-Null Count   Dtype
---  ------              --------------   -----
 0   sentiment_analysis  124368 non-null  int64
 1   release_year        124368 non-null  int32
dtypes: int32(1), int64(1)
memory usage: 2.4 MB
